ITESM CCM
Oscar Fuentes


Para correr este notebook debe tenerse familiaridad con términos de ROS como tópicos, listeners, publishers, mensajes etc.


In [17]:
import math
import numpy as np
import rospy
import time
from sensor_msgs.msg import LaserScan

from geometry_msgs.msg import Twist


In [19]:
rospy.init_node('evasion_notebook_node') 

base_vel_pub = rospy.Publisher('/cmd_vel', Twist)

/tmp/ipykernel_36669/2799142061.py:3: SyntaxWarning: The publisher should be created with an explicit keyword argument 'queue_size'. Please see http://wiki.ros.org/rospy/Overview/Publishers%20and%20Subscribers for more information.
  base_vel_pub = rospy.Publisher('/cmd_vel', Twist)


Nos valdremos de nuestra vieja función de movimiento, asi como la clase Laser para obtener las lecturas del sensor (revisar notebook 1)

Simultáneamente , es publicado en el tópico "/hsrb/head_rgbd_sensor/depth_registered/rectified_points"
, información de la nube de puntos, en forma de un mensaje tipo  PointCloud2

In [20]:

def move_base_vel(vx, vy, vw):
    twist = Twist()
    twist.linear.x = vx
    twist.linear.y = vy
    twist.angular.z = vw 
    base_vel_pub.publish(twist)

def move_base(x,y,yaw,timeout=5):
    start_time = rospy.Time.now().to_sec()
    while rospy.Time.now().to_sec() - start_time < timeout:  
        move_base_vel(x, y, yaw)
    move_base_vel(0,0, 0)

In [21]:
move_base(1,0,0,1 )

In [22]:
move_base(0,0,1.57,1)

# ODOM

In [23]:
import tf
def newOdom (msg):
    global x
    global y
    global th
    
    x=msg.pose.pose.position.x
    y=msg.pose.pose.position.y
    quaternion = (
    msg.pose.pose.orientation.x,
    msg.pose.pose.orientation.y,
    msg.pose.pose.orientation.z,
    msg.pose.pose.orientation.w)
    euler = tf.transformations.euler_from_quaternion(quaternion)
    th=euler[2]


In [24]:
#READ CLICK POINT
from geometry_msgs.msg import PointStamped
def readPoint(punto):
     global xcl , ycl
     
     xcl =punto.point.x
     ycl =punto.point.y

In [25]:
xcl,ycl

NameError: name 'xcl' is not defined

In [26]:
from nav_msgs.msg import Odometry

sub= rospy.Subscriber("/odom",Odometry,newOdom)


In [27]:
pub = rospy.Publisher('/cmd_vel', Twist, queue_size=1)

In [28]:
from sensor_msgs.msg import LaserScan

class Laser():
    u"""Class that handles laser information"""

    def __init__(self):
        # Register the _laser_cb method as a callback to the laser scan topic events
        self._laser_sub = rospy.Subscriber ('scan',
                                           LaserScan, self._laser_cb)
        self._scan_data = None

    def _laser_cb (self, msg):
        # Laser scan callback function
        self._scan_data = msg

    def get_data(self):
        u"""Function to get the laser value"""
        return self._scan_data

In [29]:
sub3=rospy.Subscriber("/clicked_point",PointStamped,readPoint)


In [30]:
xcl,ycl

NameError: name 'xcl' is not defined

# POTFIELDS

In [25]:
laser= Laser()

In [26]:
data=laser.get_data()
lec=np.asarray(data.ranges)
lec[np.isinf(lec)]=3.5


In [27]:
deltaang=   ( data.angle_max-data.angle_min  )   /len(data.ranges)

In [28]:
deltaang

0.01745330492655436

In [29]:
laserdegs=  np.arange(data.angle_min,data.angle_max,deltaang)

In [30]:
Fx=0
Fy = 0.001
for i,deg in enumerate(laserdegs):
    if i >=360:break 
    
    if (lec[i] <2.61):
        Fx = Fx + (1/lec[i])**2 * np.cos(deg)
        Fy = Fy + (1/lec[i])**2 * np.sin(deg)
         

In [31]:
Fth= np.arctan2(Fy,Fx)+np.pi
print('FxFyFth',Fx,Fy,Fth*180/np.pi)
Fmag= np.linalg.norm((Fx,Fy))

FxFyFth 1.3455555897885783 -65.91176367147114 91.16950214955668


In [32]:
xy,xycl=np.array((x,y)) ,   np.array((xcl,ycl))
euclD=np.linalg.norm(xy-xycl)
#print("xrob,yrob, throbot",x,y,th*180/3.1416)
print("xclick,yclick",xcl,ycl,"euclD",euclD)



xclick,yclick 2.4025940895080566 0.5606367588043213 euclD 4.322695295088308


In [33]:
Fatrx =( -x + xcl)/euclD
Fatry =( -y + ycl)/euclD      
Fatrth=np.arctan2(Fatry, Fatrx) 
Fatrth=Fatrth-th
Fmagat= np.linalg.norm((Fatrx,Fatry))
print ('Fatx, Fatry, Fatrth',Fatrx,Fatry,(Fatrth)*180/np.pi )

Fatx, Fatry, Fatrth 0.9961822196833766 -0.08729941872087707 -12.062393633010787


In [34]:
Ftotx= Fmag*np.cos(Fth)*.00251   +    Fmagat*np.cos(Fatrth)
Ftoty= Fmag*np.sin(Fth)*.00251    +    Fmagat*np.sin(Fatrth)
Ftotth=np.arctan2(Ftoty,Ftotx)
##################################################################
if ( Ftotth> np.pi ):Ftotth=       -np.pi-    (Ftotth-np.pi)
if (Ftotth < -np.pi):Ftotth= (Ftotth     +2 *np.pi)


In [35]:
print('Ftotxy',Ftotx,Ftoty,Ftotth*180/np.pi)


Ftotxy 0.9745433651953691 -0.043538239385566696 -2.5580182540453236


In [36]:
speed=Twist()
vel=0.4
if( abs(Ftotth) < .15) :#or (np.linalg.norm((Fx,Fy)) < 100):
     speed.linear.x=1.9
     print('lin')
     speed.angular.z=0
else:
         
        if Ftotth < 0:
            if (abs( Ftotth ) < np.pi/2):
                vel=.24
                speed.angular.z=-0.25
                print('open curve')
         
            print('Vang-')
            speed.linear.x=vel
            speed.angular.z=-0.25
        if Ftotth > 0:
            if (abs( Ftotth ) < np.pi/2):
                vel=.24
                print('open curve')
         
            
            print('Vang+')
            speed.linear.x=vel
            speed.angular.z=.25

lin


In [37]:
def get_Rep_Force():
    data=laser.get_data()
    lec=np.asarray(data.ranges)
    lec[np.isinf(lec)]=13.5
    deltaang=   ( data.angle_max-data.angle_min  )   /len(data.ranges)
    laserdegs=  np.arange(data.angle_min,data.angle_max,deltaang)
    Fx=  0.0
    Fy = 0.0
    for i,deg in enumerate(laserdegs):
        if i >=360:break 

        if (lec[i] <4.75): ###TUNABLE
            Fx = Fx + (1/lec[i])**2 * np.cos(deg)
            Fy = Fy + (1/lec[i])**2 * np.sin(deg)
    Fth= np.arctan2(Fy,(Fx))+np.pi
    #print('FxFyFth',Fx,Fy,Fth*180/np.pi)
    Fmag= np.linalg.norm((Fx,Fy))
    return Fx,Fy,Fmag,Fth
    

In [38]:
def get_Att_Force():
    xy,xycl=np.array((x,y)) ,   np.array((xcl,ycl))
    euclD=np.linalg.norm(xy-xycl)
    #print("xrob,yrob, throbot",x,y,th*180/3.1416)
    #print("xclick,yclick",xcl,ycl,"euclD",euclD)

    Fatrx =( -x + xcl)/euclD
    Fatry =( -y + ycl)/euclD      
    Fatrth=np.arctan2(Fatry, Fatrx) 
    Fatrth=Fatrth-th
    Fmagat= np.linalg.norm((Fatrx,Fatry))
    #print ('Fatx, Fatry, Fatrth',Fatrx,Fatry,(Fatrth)*180/np.pi )
    return  Fatrx, Fatry , Fmagat, Fatrth, euclD

In [39]:
def get_Speed(Ftotx,Ftoty,Ftotth):
    speed=Twist()
    if( abs(Ftotth) < .1) :
        
        speed.linear.x=0.13
        #print('lin')
        speed.angular.z=0

    else:
        
        if Ftotth < 0:                            
                if (abs( Ftotth ) < np.pi/2):
                    speed.linear.x= 0.025
                    speed.angular.z=-0.25
                    #print('open curve')
                
                else:                    
                    #print('Vang-')
                    speed.linear.x=  0.0
                    speed.angular.z=-0.5
                        
        if Ftotth > 0:
                if (abs( Ftotth ) < np.pi/2):
                    speed.linear.x= 0.05
                    speed.angular.z=.25
                    #print('open curve')
                
                else:
                    #print('Vang+')
                    speed.linear.x=  0.0
                    speed.angular.z= 0.5
    return speed

In [40]:
get_Speed(Ftotx,Ftoty,Ftotth)

linear: 
  x: 0.13
  y: 0.0
  z: 0.0
angular: 
  x: 0.0
  y: 0.0
  z: 0

In [ ]:
rate = rospy.Rate(10)
counter=0
while not rospy.is_shutdown():
        counter+=1
        Fx,Fy,Fmag,Fth          =get_Rep_Force()
        Fatx,Faty,Fmagat,Fatrth, euclD =get_Att_Force()
        #######TUNABLE
        Ftotx= Fmag*np.cos(Fth)*.004   +    Fmagat*np.cos(Fatrth)
        Ftoty= Fmag*np.sin(Fth)*.004    +    Fmagat*np.sin(Fatrth)
        Ftotth=np.arctan2(Ftoty,Ftotx)
        ###ANGLE NORMALIZATION
        if ( Ftotth> np.pi ):     Ftotth=       -np.pi-    (Ftotth-np.pi)
        if (Ftotth < -np.pi): Ftotth= (Ftotth     +2 *np.pi)
        if counter==1000000:
            print('Ftotxy',Ftotx,Ftoty,Ftotth*180/np.pi, 'eulcD', euclD)
            counter=0
        ####
        speed=get_Speed(Ftotx,Ftoty,Ftotth)
        #if eculD < 2: speed.linear.x=0.2
        #if eculD < 0.5: 
            #speed.linear.x=0 
            #print("im here", euclD,(eculD < 0.5))
            #rospy.sleep(1)
            #break
        pub.publish(speed)
        rate.sleep()

In [ ]:
# ENTREGAR

# TAREA POR EQUIPOS

# Programar al menos 3 conductas para el turltebot. 
# UTILIZAR MAQUINAS DE ESTADOS  y SMACH
 
 ## 1.   De ellas será  una evasión de obstáculos modificada de la tarea 1 para esta vez "rodear" el obstáculo
 (I.A. Clásica)
 
 ## 2.  Implemente navegación reactiva utilizando campos potenciales y SLAM
 (Completamente Reactivo).
 
 ## 3.  Implemente un "árbtiro" que sea capaz de detectar que el agente se encuentra en un mínimo local y repcuperar la movilidad.(Híbrido)
 
 ## 4. Utililzando tf2_ros  Obtenga un estimado de las coordenadas más "seguras".
 ### se sabe que las patas de las  mesas un son punto particularmente peligroso...
 
 Publique una transformada (tf ) en posiciones cuyas lecturas de lidar permiten suponer la presencia de una mesa o silla ... patas .
 
 
 
 
 
 
 

# Obtener la ubicación del agente en el mapa... MAPAS SLAM:... 

In [ ]:
import tf2_ros
tfBuffer = tf2_ros.Buffer()
listener = tf2_ros.TransformListener(tfBuffer)
def get_coords ():
    trans = tfBuffer.lookup_transform('map', 'base_link', rospy.Time())
    return trans


In [ ]:
t=get_coords()
type (t)

In [ ]:
broadcaster = tf2_ros.StaticTransformBroadcaster()

In [ ]:
t.child_frame_id='here'

In [ ]:
broadcaster = tf2_ros.StaticTransformBroadcaster()


In [ ]:
broadcaster.sendTransform(t)